In [ ]:
import pandas as pd
import numpy as np

# Load your data
url = '/content/dataset.csv'
data = pd.read_csv(url)


In [ ]:

# Convert the 'Date Time' column to datetime
#data['Date Time'] = pd.to_datetime(data['Date Time'])

# Define the breakpoints and index values from the AQI guidelines
breakpoints = {
    'CO': [(0, 10000, 0, 50), (10000, 30000, 50, 100), (30000, 45000, 100, 150), (45000, 60000, 150, 200), (60000, 90000, 200, 300)],
    'PM25': [(0, 25, 0, 50), (25, 50, 50, 100), (50, 80, 100, 150), (80, 150, 150, 200),(150, 250, 200, 300), (250, 350, 300, 400) ],
    'PM10': [(0, 50, 0, 50), (50, 150, 50, 100), (150, 250, 100, 150), (250, 350, 150, 200),(350, 420, 200, 300), (420, 500, 300, 400) ],
}

# Function to calculate AQI based on concentration
def calculate_aqi(C, bp_low, bp_high, I_low, I_high):
    return ((I_high - I_low) / (bp_high - bp_low)) * (C - bp_low) + I_low


# Calculate Nowcast for PM2.5
def calculate_nowcast(data):
    nowcast_values = []
    for i in range(len(data)):
        if i < 11:  # Not enough previous data to calculate Nowcast
            nowcast_values.append(np.nan)
            continue

        # Select the last 12 values including the current hour
        recent_data = data.iloc[i-11:i+1]['PM25']
        C_max = recent_data.max()
        C_min = recent_data.min()

        # Avoid division by zero and ensure w is meaningfully calculated
        if C_min == 0:  # Avoid division by zero
            w = 1
        else:
            w = C_min / C_max

        # Set the minimum value for w to ensure a sensible decay rate
        if w <= 0.5:
            w = 0.5

        # Calculate weights in reverse order so the oldest data has the highest exponent
        weights = np.power(w, np.arange(0, 12))  # w^0, w^1, ..., w^11

        # Compute the nowcast using the generated weights, older data is more weighted
        nowcast = np.sum(weights[::-1] * recent_data) / np.sum(weights)
        nowcast_values.append(nowcast)

    return nowcast_values

# Calculate Nowcast values for PM2.5 and add to dataframe
data['Nowcast_PM25'] = calculate_nowcast(data)

# Calculate Nowcast for PM10
def calculate_nowcastPM10(data):
    nowcast_values = []
    for i in range(len(data)):
        if i < 11:  # Not enough previous data to calculate Nowcast
            nowcast_values.append(np.nan)
            continue

        # Select the last 12 values including the current hour
        recent_data = data.iloc[i-11:i+1]['PM10']
        C_max = recent_data.max()
        C_min = recent_data.min()

        # Avoid division by zero and ensure w is meaningfully calculated
        if C_min == 0:  # Avoid division by zero
            w = 1
        else:
            w = C_min / C_max

        # Set the minimum value for w to ensure a sensible decay rate
        if w <= 0.5:
            w = 0.5

        # Calculate weights in reverse order so the oldest data has the highest exponent
        weights = np.power(w, np.arange(0, 12))  # w^0, w^1, ..., w^11

        # Compute the nowcast using the generated weights, older data is more weighted
        nowcast = np.sum(weights[::-1] * recent_data) / np.sum(weights)
        nowcast_values.append(nowcast)

    return nowcast_values

# Calculate Nowcast values for PM2.5 and add to dataframe
data['Nowcast_PM10'] = calculate_nowcastPM10(data)

# Function to determine which breakpoints to use
def get_aqi(C, pollutant):
    for bp in breakpoints[pollutant]:
        if bp[0] <= C <= bp[1]:
            return calculate_aqi(C, *bp)
    return np.nan  # Return NaN if the concentration is out of range


# Calculate hourly AQI for each pollutant
data['AQI_CO(h)'] = data['CO'].apply(get_aqi, pollutant='CO')
data['AQI_PM25(h)'] = data['Nowcast_PM25'].apply(get_aqi, pollutant='PM25')
data['AQI_PM10(h)'] = data['Nowcast_PM10'].apply(get_aqi, pollutant='PM10')

# Save the updated DataFrame to a new CSV file
data.to_csv('updated_aqi_values.csv', index=False)

# Display the first few rows of the updated dataframe
print(data.head())


   AQI     CO      Date Time  PM10  PM25           Local Time  Nowcast_PM25  \
0  131  170.6  2023-03-07:17  58.8  47.0  2023-03-08T00:00:00           NaN   
1  126  158.7  2023-03-07:18  56.3  45.0  2023-03-08T01:00:00           NaN   
2  118  150.2  2023-03-07:19  52.5  42.0  2023-03-08T02:00:00           NaN   
3  126  141.6  2023-03-07:20  56.3  45.0  2023-03-08T03:00:00           NaN   
4  118  133.1  2023-03-07:21  52.5  42.0  2023-03-08T04:00:00           NaN   

   AQI_CO(h)  AQI_PM25(h)  Nowcast_PM10  AQI_PM10(h)  
0     0.8530          NaN           NaN          NaN  
1     0.7935          NaN           NaN          NaN  
2     0.7510          NaN           NaN          NaN  
3     0.7080          NaN           NaN          NaN  
4     0.6655          NaN           NaN          NaN  
